In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
#geopy installation

!pip install geopy
import geopy 

from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim
import time
from pprint import pprint
geolocatorGoogle = GoogleV3(api_key=" ")
geolocatorNom = Nominatim(user_agent="cphDorms")

In [3]:
#Using hashlib to shorten dorm names and make them identifiable (Some Dorm Names Suck!)
import base64
from hashlib import blake2b


def hash(input):
    hasher = blake2b(digest_size=4)
    hasher.update(input.encode())
    return str(base64.urlsafe_b64encode(hasher.digest()).decode('UTF-8')).rstrip('=')

hash("Albertslund UngdomsboligerMorbærhaven 0 , 2620 Albertslund")

'J4nBfw'

In [4]:
url = "https://www.kollegierneskontor.dk/default.aspx?func=kkikportal.kollegiumlist&mid=40&topmenuid=34&lang=GB"
results = requests.get(url)
soup = BeautifulSoup(results.text, "html.parser")

In [5]:
#Longitude and latitude

def coordinatesGoogle(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorGoogle.geocode(address)
             return {"latitude":float(location.latitude), "longitude":float(location.longitude)}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":-1, "longitude":-1}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location

#print(f"{latitude}, {longitude}")
# print all returned data
#pprint(location)

def coordinatesNomantim(dorm_address):
    def get_location_by_address(address):
        #Short Wait
        try:
             location = geolocatorNom.geocode(address).raw
             return {"latitude":float(location['lat']), "longitude":float(location['lon'])}
        except:
            print("Couldn'd find coordinates of " + address)
            return {"latitude":-1, "longitude":-1}
    location = get_location_by_address(dorm_address)
    #latitude = location["latitude"]
    #longitude = location["longitude"]
    return location


print(coordinatesNomantim("Dalslands,hjhkljhgade 8, Copenhagen"))

Couldn'd find coordinates of Dalslands,hjhkljhgade 8, Copenhagen
{'latitude': -1, 'longitude': -1}


In [6]:

kkik = []

kollegium = soup.find_all('div', class_='kollegium row')


for container in kollegium:
    print("Processing Dorm")
    dorm_name = container.h3.a.text
    dorm_address = container.div.find('div', class_="address").text.strip().replace("\r\n               ", "").replace("\xa0", "")
    try:
        dorm_picture_link = container.find('img', class_="kollegiumlistimage img")['src']
    except: 
        dorm_picture_link = ""

    body = container.find('div', class_="body col-xs-12")

    description = list(map(str.strip,body.find('div', class_="description").find_all(text=True, recursive=True)))
    

    rows = body.find_all("div", class_= "row")
    rows.pop(0)
    rows.pop(0)
    rows.pop(0)

    rooms_export = []

    for room in rows:
        room_details = room.find_all("div")

        roomtype = room_details[0].get_text().strip()
        size = room_details[1].get_text().strip()
        size = float(re.split(r' |\(|/|-|m',size)[0].replace(",","."))
        price = float(room_details[3].get_text().strip().split(" ")[1].replace(",",""))
        
        rooms_export.append({
            "roomType":roomtype,
            "size":size,
            "price":price
            
        })

    kkik.append({
        "name":dorm_name,
        "address":dorm_address,
        "uuid":hash(dorm_name + dorm_address),
        "dormPictureLink":"https://www.kollegierneskontor.dk/"+dorm_picture_link,
        "coordinates":coordinatesNomantim(dorm_address),
        "description":description,
        "rooms":rooms_export
    })

    print(dorm_name + dorm_address)
    print(hash(dorm_name + dorm_address))

    print()






Processing Dorm
Albertslund UngdomsboligerMorbærhaven 0 , 2620 Albertslund
J4nBfw

Processing Dorm
Amagerbrogade 268Amagerbrogade 268 , 2300 København S
KMLY4w

Processing Dorm
Dahlerup UngdomsboligerDahlerupsgade 1 , 1603 København V
auJZ6w

Processing Dorm
Danmarks Internationale KollegiumVognporten 14 , 2620 Albertslund
5DdXUA

Processing Dorm
Couldn'd find coordinates of Emdrupvej 54 A, 2400 København NV
Det Nye Emdrupborg Kollegium (administreres af FA09)Emdrupvej 54 A, 2400 København NV
G-z6og

Processing Dorm
Falkoner Kollegiet (administreres og udlejes af WA ApS)Sylows Allé 13 , 2000 Frederiksberg
EaVmRg

Processing Dorm
Frankrigsgade KollegietFrankrigsgade 50 , 2300 København S
_p3oFQ

Processing Dorm
Grønjordskollegiet (administreres af FA09)Grønjordsvej 1 , 2300 København S
RwONEw

Processing Dorm
Hedorfs KollegiumPreben Kaas' Vænge 5 , 2000 Frederiksberg
j-bJLA

Processing Dorm
Couldn'd find coordinates of Holmbladsgade 70 C, 2300 København S
Holmbladsgade 70CHolmbladsgade 

In [7]:
# Add Dorm Min&Max Price
for index, dorm in enumerate(kkik):
    kkik[index]["maxPrice"] = 0
    kkik[index]["minPrice"] = float('inf')
    kkik[index]["maxSize"] = 0
    kkik[index]["minSize"] = float('inf')
    for room in dorm["rooms"]:
        kkik[index]["maxPrice"] = max(kkik[index]["maxPrice"], room["price"])
        kkik[index]["minPrice"] = min(kkik[index]["minPrice"], room["price"])
        kkik[index]["maxSize"] = max(kkik[index]["maxSize"], room["size"])
        kkik[index]["minSize"] = min(kkik[index]["minSize"], room["size"])


In [8]:
import json

with open('json_kkik.json', 'w') as f:
    json.dump(kkik, f)


ValueError: could not convert string to float: '41,5 m2'